In [1]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

import copy

from pof.failure_mode import FailureMode
from interface.layouts import *

In [2]:
fm = FailureMode().set_demo()

In [3]:
# Component Tests

mcl = make_component_layout(comp, prefix='comp-')

# Failure Mode Tests
mfml = make_failure_mode_layout(fm, prefix="fm-")
"""
# Task Tests
task = fm.tasks['ocr']
mtl = make_task_layout(task)

mtf = make_task_form(task, prefix="")

# Trigger Tests
triggers= fm.tasks['ocr'].triggers
mttl = make_task_trigger_layout(triggers)

condition_triggers = fm.tasks['ocr'].triggers['condition']
#mctf = make_condition_trigger_form(condition_triggers)


# Impact Tests

impacts= fm.tasks['ocr'].impacts
mtil = make_task_impact_layout(impacts)

state_impacts = fm.tasks['ocr'].impacts['state']
msil = make_state_impact_layout(state_impacts)

condition_impacts = fm.tasks['ocr'].impacts['condition']
mcil = make_condition_impact_layout(condition_impacts)

condition_impact = fm.tasks['ocr'].impacts['condition']['wall_thickness']
mtif = make_condition_impact_form(condition_impact)

# Distribution Tests
failure_dist = fm.failure_dist
mdl = make_dist_layout(failure_dist)"""

'\n# Task Tests\ntask = fm.tasks[\'ocr\']\nmtl = make_task_layout(task)\n\nmtf = make_task_form(task, prefix="")\n\n# Trigger Tests\ntriggers= fm.tasks[\'ocr\'].triggers\nmttl = make_task_trigger_layout(triggers)\n\ncondition_triggers = fm.tasks[\'ocr\'].triggers[\'condition\']\n#mctf = make_condition_trigger_form(condition_triggers)\n\n\n# Impact Tests\n\nimpacts= fm.tasks[\'ocr\'].impacts\nmtil = make_task_impact_layout(impacts)\n\nstate_impacts = fm.tasks[\'ocr\'].impacts[\'state\']\nmsil = make_state_impact_layout(state_impacts)\n\ncondition_impacts = fm.tasks[\'ocr\'].impacts[\'condition\']\nmcil = make_condition_impact_layout(condition_impacts)\n\ncondition_impact = fm.tasks[\'ocr\'].impacts[\'condition\'][\'wall_thickness\']\nmtif = make_condition_impact_form(condition_impact)\n\n# Distribution Tests\nfailure_dist = fm.failure_dist\nmdl = make_dist_layout(failure_dist)'

In [4]:

# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Layout
app.layout = html.Div([
    html.Div(children='Test Output', id='test_output'),
    dcc.Graph(id="maintenance_strategy"),
    mfml,
])

sep= '-'
collapse_ids = []
for fm_name, fm in {'fm': fm}.items():
    collapse_ids = collapse_ids + [fm_name]
    for task in fm.tasks:
        collapse_ids = collapse_ids + [fm_name + sep + 'task' + sep + task]

@app.callback(
    #Output("test_output", "children"),
    [Output(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
    [Input(f"{prefix}-collapse-button", "n_clicks") for prefix in collapse_ids],
    [State(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
)
def toggle_collapses(*args):
    ctx = dash.callback_context

    state_id = ""
    collapse_id = ctx.triggered[0]['prop_id'].split('.')[0].replace('-collapse-button','')
    if collapse_id in collapse_ids: #TODO change to is not None

        state_id = collapse_id + '-collapse.is_open'
        ctx.states[state_id] = not ctx.states[state_id]

    is_open = tuple(ctx.states.values())

    return is_open

ms_fig_update = fm.get_dash_ids(prefix ='fm-')

@app.callback(
    Output("maintenance_strategy", "figure"), 
    [Input(dash_id,"checked") if 'active' in dash_id else Input(dash_id,"value") for dash_id in ms_fig_update]
)
def update_maintenance_strategy(*args):

    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
    value = ctx.triggered[0]['value']

    #Temp utnil mutliple failure modes are set up
    dash_id = dash_id.replace('fm-', "")

    # Update the model
    fm_local = copy.deepcopy(fm)
    fm_local.dash_update(dash_id, value)

    fm_local.mc_timeline(t_end=200, n_iterations=100)
    df = fm_local.expected_cost_df()

    fig = px.area(
        df,
        x="time",
        y="cost_cumulative",
        color="task",
        title="Maintenance Strategy Costs",
    )

    #fig.update_yaxes(range=[0, graph_y_limit])

    return fig

@app.callback(
    Output("test_output", "children"),
    [Input('fm-active', 'checked')]
)
def test(task_value):
    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
    value = ctx.triggered[0]['value']

    return "value" + str(value)

#Execute
app.run_server(mode='inline', debug=True)

In [8]:
'----'.split(sep)[0]

''

In [6]:
fm.dash_update('task-inspection-active', False)
#fm.dash_update('failure_dist-alpha', 34)
#fm.dash_update('active', False)

In [7]:
dash_id = 'task-inspection-active'
value = True

try:
    
    next_id = dash_id.split(sep)[0]

    # Check if the next component is a param of 
    if next_id in ['active']:

        fm.active = value

    elif next_id == 'failure_dist':

        dash_id = dash_id.replace(next_id + sep, "")
        fm.failure_dist.dash_update(dash_id, value)

    elif next_id == 'task':
        dash_id = dash_id.replace(next_id + sep, "")
        task_name= dash_id.split(sep)[0]
        dash_id = dash_id.replace(task_name + sep, "")
        fm.tasks[task_name].dash_update(dash_id, value)

except:

    print("Invalid Dash ID %s" %(dash_id))

In [8]:
dash_id = 'task-inspection-active'
dash_id = dash_id.replace(next_id + sep, "")
task_name= dash_id.split(sep)[0]
dash_id = dash_id.replace(next_id + sep, "")
dash_id

'inspection-active'

In [13]:
task.active

False

In [12]:
fm.tasks['inspection'].__dict__['active'] = False

In [11]:
fm.tasks['inspection'].__dict__

{'name': 'unknown',
 'activity': 'inspection',
 'trigger': 'time',
 'active': True,
 'trigger_comp': '1.1.2',
 '_package': NotImplemented,
 '_impacts_parent': NotImplemented,
 '_impacts_children': False,
 'cost': 50,
 'labour': 'trade',
 'spares': 'pole',
 'equipment': 'ewp',
 'consequence': None,
 'p_effective': 0.91,
 'triggers': {'condition': {'wall_thickness': {'lower': 0, 'upper': 90}},
  'state': {'initiation': True}},
 'impacts': {'condition': {}, 'state': {'detection': True}},
 'component_reset': False,
 'state': 'up',
 't_completion': [5,
  10,
  15,
  20,
  25,
  30,
  35,
  40,
  45,
  50,
  55,
  60,
  65,
  70,
  75,
  80,
  85,
  90,
  95,
  100,
  105,
  110,
  115,
  120,
  125,
  130,
  135,
  140,
  145,
  150,
  155,
  160,
  165,
  170,
  175,
  180,
  185,
  190,
  195,
  200,
  5,
  10,
  15,
  20,
  25,
  30,
  35,
  40,
  45,
  50,
  55,
  60,
  65,
  70,
  75,
  80,
  85,
  90,
  95,
  100,
  105,
  110,
  115,
  120,
  125,
  130,
  135,
  140,
  145,
  150,
 